<h1>Comparative Dataframe</h1>
<h2> Notebook com função em spark para analisar divergências entre dois notebooks</h2>

## 0.0 Imports

In [0]:
%python

from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [0]:
df1 = spark.createDataFrame([
  [1, "KKK", 5000, "US"],
  [2, "DEF", 4000, "UK"],
  [3, "GHI", 3000, "JPN"],
  [4, "JKL", 4500, "CHN"]
], ["id", "name", "sal", "Address"])

df2 = spark.createDataFrame([
  [1, "ABC", 5000, "US"],
  [2, "HGT", 4000, "CAN"],
  [3, "GHI", 3500, "JPN"],
  [4, "JKL_M", 4800, "CHN"]
], ["id", "name", "sal", "Address"])

#Function 1 - Just Check if the dataframes are the same

In [0]:
def are_dfs_equal(df1, df2):
    if df1.schema != df2.schema:
        return False
    if df1.collect() != df2.collect():
        return False
    return True
  
spark.udf.register('are_dfs_equal', are_dfs_equal)

Out[73]: <function __main__.are_dfs_equal(df1, df2)>

In [0]:
are_dfs_equal(centroid_demographic_segmentation, chs_centroid_all_dates)

Out[74]: False

# Function2 - Spark Builtin function - Shows the different cells between the 2 dataframes

In [0]:
subtract_function = df1.subtract(df2)
display(subtract_function)

id,name,sal,Address
3,GHI,3000,JPN
4,JKL,4500,CHN
1,KKK,5000,US
2,DEF,4000,UK


# Function3 - compare dataframes, and brings different values for each row

In [0]:
def df_compare(df1, df2, key_value):
  conditions_ = [when(df1[c]!=df2[c], lit(c)).otherwise("") for c in df1.columns if c != key_value]

  select_expr =[
                col(key_value), 
                *[df2[c] for c in df2.columns if c != key_value], 
                array_remove(array(*conditions_), "").alias("column_names")
]

  df3 = df1.join(df2, key_value).select(*select_expr)
  return df3
spark.udf.register('df_compare', df_compare)

Out[76]: <function __main__.df_compare(df1, df2, key_value)>

In [0]:
df_teste = df_compare(df1, df2, 'id')

In [0]:
display(df_teste)

id,name,sal,Address,column_names
1,ABC,5000,US,List(name)
2,HGT,4000,CAN,"List(name, Address)"
3,GHI,3500,JPN,List(sal)
4,JKL_M,4800,CHN,"List(name, sal)"
